In [1]:
from __future__ import print_function, division
import numpy as np
from os.path import join, expanduser
import matplotlib.pyplot as plt
import yaml  # for pretty-printing dict
from neuralnilm.metrics import run_metrics, across_all_appliances
import pandas as pd

# sklearn evokes warnings from numpy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/jack/workspace/python/Lasagne/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "
/usr/local/lib/python2.7/dist-packages/sklearn/utils/__init__.py:11: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility
  from .murmurhash import murmurhash3_32
/usr/local/lib/python2.7/dist-packages/sklearn/utils/sparsetools/__init__.py:3: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility
  from ._min_spanning_tree import minimum_spanning_tree
/usr/local/lib/python2.7/dist-packages/sklearn/utils/sparsetools/_graph_validation.py:5: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility
  from ._graph_tools import csg

In [107]:
TRAIN_HOUSES = {
    'microwave': (1, 2),
    'fridge': (1, 2, 4),
    'dish washer': (1, 2),
    'kettle': (1, 2, 4),
    'washing machine': (1, 5)
}

TEST_HOUSES = {
    'microwave': (5,),
    'fridge': (5,),
    'dish washer': (5,),
    'kettle': (5,),
    'washing machine': (2,)
}

APPLIANCES = TRAIN_HOUSES.keys()

ON_POWER_THRESHOLDS = {
    'microwave': 200,
    'fridge': 50,
    'dish washer': 10,
    'kettle': 2000,
    'washing machine': 20
}

HOUSES = [1, 2, 3, 4, 5]

METRICS = [
    'f1_score',
    'precision_score',
    'recall_score',
    'accuracy_score',
    'relative_error_in_total_energy',
    'total_energy_correctly_assigned',
    'mean_absolute_error'
]

# ALGORITHMS = ['co', 'fhmm', 'ae', 'rectangles', 'rnn']

ALGORITHMS = ['co', 'fhmm', 'ae', 'rectangles']

full_algorithm_names = [
    'Combinatorial Optimisation ', 'Factorial HMM', 'Autoencoder', 'Rectangles']


ESTIMATES_PATH = expanduser(
    "~/PhD/experiments/neural_nilm/data_for_BuildSys2015/disag_estimates")
GROUND_TRUTH_PATH = expanduser(
    "~/PhD/experiments/neural_nilm/data_for_BuildSys2015/ground_truth_and_mains")

In [ ]:
# TODO mean and zero

In [3]:
def load(architecture, building_i, appliance):
    # load estimates
    estimates_fname = "{}_building_{}_estimates_{}.csv".format(
        architecture, building_i, appliance)
    estimates_fname = join(ESTIMATES_PATH, estimates_fname)
    y_pred = np.loadtxt(estimates_fname, delimiter=',')

    # load ground truth
    y_true_fname = "building_{}_{}.csv".format(building_i, appliance.replace(' ', '_'))
    y_true_fname = join(GROUND_TRUTH_PATH, y_true_fname)
    y_true = np.loadtxt(y_true_fname, delimiter=',')

    # load mains
    mains_fname = "building_{}_mains.csv".format(building_i)
    mains_fname = join(GROUND_TRUTH_PATH, mains_fname)
    mains = np.loadtxt(mains_fname, delimiter=',')

    return y_true, y_pred, mains

In [4]:
def plot_all(y_true, y_pred, mains, title=None):
    fig, axes = plt.subplots(nrows=3, sharex=True)
    axes[0].plot(y_pred)
    axes[0].set_title('y_pred')
    axes[1].plot(y_true)
    axes[1].set_title('y_true')
    axes[2].plot(mains)
    axes[2].set_title('mains')
    if title:
        fig.set_title(title)
    plt.show()
    return fig, axes

(<matplotlib.figure.Figure at 0x7fd949085350>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7fd948e6d990>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x7fd9484ef9d0>], dtype=object))

In [6]:
# Run metrics


In [93]:
def calc_metrics(houses):
    scores = pd.Panel(
        np.NaN,
        items=APPLIANCES,
        major_axis=METRICS,
        minor_axis=ALGORITHMS
    )
    
    for appliance in APPLIANCES:
        houses_for_appliance = houses[appliance]
        on_power_threshold = ON_POWER_THRESHOLDS[appliance]
        for algo in ALGORITHMS:
            house_scores = pd.DataFrame(
                np.NaN, columns=METRICS, index=houses_for_appliance)
            for house_i in houses_for_appliance:
                y_true, y_pred, mains = load(algo, house_i, appliance)
                house_scores_dict = run_metrics(
                    y_true, y_pred, mains, on_power_threshold)
                house_scores_dict.pop('sum_abs_diff')
                house_scores.loc[house_i] = house_scores_dict
            scores[appliance, :, algo].update(house_scores.dropna().mean())
                
    return scores

In [94]:
test_houses_scores = calc_metrics(TEST_HOUSES)
train_houses_scores = calc_metrics(TRAIN_HOUSES)

In [105]:
APPLIANCE = 'fridge'
test_houses_scores[APPLIANCE]

,co,fhmm,ae,rectangles
f1_score,0.345734,0.549839,0.000000,0.684371
precision_score,0.300388,0.403669,0.000000,0.605539
recall_score,0.407204,0.861959,0.000000,0.786800
accuracy_score,0.450810,0.497060,0.643657,0.741389
relative_error_in_total_energy,0.371471,0.566832,-0.528532,-0.016614
total_energy_correctly_assigned,0.940019,0.944801,0.963745,0.975478
mean_absolute_error,73.275903,67.433414,44.290833,29.957072


In [106]:
train_houses_scores[APPLIANCE]

,co,fhmm,ae,rectangles
f1_score,0.520795,0.471406,0.000000,0.544079
precision_score,0.499031,0.385641,0.000000,0.578835
recall_score,0.544940,0.628293,0.000000,0.518748
accuracy_score,0.611349,0.463754,0.614414,0.667419
relative_error_in_total_energy,0.262454,0.495344,-0.514447,-0.267002
total_energy_correctly_assigned,0.937376,0.911494,0.946837,0.958188
mean_absolute_error,49.750708,69.067558,40.582481,32.214409


In [98]:
y_true, y_pred, mains = load('rectangles', 2, APPLIANCE)
plot_all(y_true, y_pred, mains)

(<matplotlib.figure.Figure at 0x7fd947a7de10>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7fd947a75dd0>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x7fd9479cde10>], dtype=object))

In [146]:
# plot
COLOR = ['#5F7343', '#99A63C', '#FEC06A', '#F25430', '#E61924']
nrows = len(METRICS)
ncols = len(APPLIANCES)
n_algorithms = len(ALGORITHMS)
x = range(n_algorithms)
FONTSIZE = 10

def plot_scores(scores):
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharey='row', figsize=(8.1, 8.7))
    fig.patch.set_facecolor('white')
    for row_i, metric in enumerate(METRICS):
        for col_i, appliance in enumerate(APPLIANCES):
            ax = axes[row_i, col_i]
            scores_for_algorithms = scores[appliance, metric]
            rects = ax.bar(
                x, scores_for_algorithms, color=COLOR, edgecolor=COLOR, zorder=3)

            # Numbers on the plot
            if row_i == 6:  # mean absolute error (watts)
                text_y = 150
                text_format = '{:3.0f}'
            else:
                text_y = 0.5
                text_format = '{:.2f}'

            # Draw text
            for i, rect in enumerate(rects):
                ax.text(
                    rect.get_x() + rect.get_width() / 2.5,
                    text_y,
                    text_format.format(scores_for_algorithms[i]),
                    va='center', rotation=90, fontsize=FONTSIZE)

            # Formatting
            ax.set_xticks([])
            ax.tick_params(direction='out')
            ax.yaxis.grid(
                b=True, which='major', color='white', linestyle='-', zorder=0)
            ax.patch.set_facecolor((0.85, 0.85, 0.85))

            if row_i == 4:  # relative error in total energy
                ax.set_ylim((-1, 1))

            for spine in ['top', 'right', 'left', 'bottom']:
                ax.spines[spine].set_visible(False)

            if row_i == 0:
                if appliance == 'across all appliances':
                    label = 'Across all\nappliances'
                else:
                    label = appliance.replace(' ', '\n')
                    label = label[0].capitalize() + label[1:]
                ax.set_title(label, fontsize=FONTSIZE)
            if col_i == 0:
                label = metric.replace('_', '\n')
                if label == 'mean\nabsolute\nerror':
                    label = label + '\n(watts)'
                elif label == 'total\nenergy\ncorrectly\nassigned':
                    label = 'prop. of\n' + label
                elif label == 'relative\nerror\nin\ntotal\nenergy':
                    label = 'relative\nerror in\ntotal\nenergy'
                label = label[0].capitalize() + label[1:]
                ylabel = ax.set_ylabel(label, fontsize=FONTSIZE)
                ylabel.set_rotation('horizontal')
                ylabel.set_verticalalignment('center')
                ylabel.set_horizontalalignment('center')
                ax.yaxis.labelpad = 25
                ax.tick_params(axis='y', left='on', right='off')
            else:
                ax.tick_params(axis='y', left='off', right='off')

    plt.subplots_adjust(hspace=0.3)
    plt.legend(rects, full_algorithm_names, ncol=n_algorithms, loc=(-5, -0.8),
               frameon=False)
    return fig, axes

In [148]:
fig, axes = plot_scores(test_houses_scores)
#fig.suptitle('Unseen houses', fontsize=16)
plt.show()

In [149]:
fig, axes = plot_scores(train_houses_scores)
#fig.suptitle('Train houses', fontsize=16)
plt.show()